In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [4]:
!kaggle datasets download -d salader/dogs-vs-cats

 99% 1.05G/1.06G [00:09<00:00, 227MB/s]
100% 1.06G/1.06G [00:09<00:00, 115MB/s]


In [5]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [6]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Flatten
from keras.applications.vgg16 import VGG16

In [7]:
conv_base = VGG16(
    weights='imagenet',
    include_top = False,
    input_shape=(150,150,3)
)


58889256/58889256 [==============================] - 0s 0us/step


In [8]:
model = Sequential()

model.add(conv_base)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [9]:
conv_base.trainable = False


In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [13]:
batch_size = 32

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '/content/train',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        '/content/test',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [16]:
print(len(train_generator),len(validation_generator))

625 157


This code snippet is using Keras' `ImageDataGenerator` to perform data augmentation for training a Convolutional Neural Network (CNN) for image classification. Data augmentation involves applying various transformations to the training images to artificially increase the diversity of the training dataset, which can help improve the model's ability to generalize.

Let's break down the code step by step:

```python
batch_size = 32

# Define the data augmentation configuration for training images
train_datagen = ImageDataGenerator(
    rescale=1./255,          # Rescale pixel values to [0, 1]
    shear_range=0.2,         # Apply shear transformations
    zoom_range=0.2,          # Apply zoom transformations
    horizontal_flip=True     # Apply horizontal flips
)

# Define the data augmentation configuration for validation/test images
test_datagen = ImageDataGenerator(rescale=1./255)

# Create data generators for training and validation/test sets
train_generator = train_datagen.flow_from_directory(
    '/content/train',         # Path to training images
    target_size=(150, 150),   # Resize images to (150, 150)
    batch_size=batch_size,    # Batch size for training
    class_mode='binary'       # Binary classification task
)

validation_generator = test_datagen.flow_from_directory(
    '/content/test',          # Path to validation/test images
    target_size=(150, 150),   # Resize images to (150, 150)
    batch_size=batch_size,    # Batch size for validation/test
    class_mode='binary'       # Binary classification task
)
```

Explanation of key parts:

1. `ImageDataGenerator`: This class in Keras provides a range of image augmentation techniques that can be applied to the training data.

2. `train_datagen`: An instance of `ImageDataGenerator` is created for the training images. It includes several augmentation techniques:
   - `rescale`: Rescales pixel values to the range [0, 1].
   - `shear_range`: Applies shear transformations to the images.
   - `zoom_range`: Applies zoom transformations to the images.
   - `horizontal_flip`: Applies horizontal flips to the images.

3. `test_datagen`: An instance of `ImageDataGenerator` is created for the validation/test images. It only includes pixel value rescaling.

4. `train_generator` and `validation_generator`: These are data generator objects created using the `flow_from_directory` method. They generate batches of augmented images on-the-fly during training and validation. Images are resized to (150, 150) pixels and batched based on the `batch_size`.

As for your question about whether the new generated data will be stored in another folder or in the `train_generator` or `validation_generator` variables, the augmented data is not physically stored in separate folders or variables. It's generated on-the-fly as batches are requested during training and validation. Each time you request a batch from `train_generator` or `validation_generator`, the images are augmented based on the specified transformations. This dynamic augmentation process improves efficiency by avoiding the need to store the augmented images separately.

In [17]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [18]:
history = model.fit_generator(
        train_generator,
        epochs=5,
        validation_data=validation_generator);

<ipython-input-18-e51005dcbc31>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/5
 11/625 [..............................] - ETA: 1:32:51 - loss: 1.3396 - accuracy: 0.5170

KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'],color='red',label='train')
plt.plot(history.history['val_accuracy'],color='blue',label='validation')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'],color='red',label='train')
plt.plot(history.history['val_loss'],color='blue',label='validation')
plt.legend()
plt.show()